In [1]:
#### Project 

In [20]:
#### Import packages
from bs4 import BeautifulSoup
import urllib3
import re
import time
from collections import defaultdict
import numpy as np
import tqdm

In [21]:
### Let's define the urlpage
urlpage = 'https://www.premierleague.com/tables'

In [22]:
### Let's define a function to get the web page and subtract the part of the html codes we need (function n°1)

def get_page(urlpage,element,html_id):
    # Get page in html
    req = urllib3.PoolManager()
    res = req.request('GET', urlpage)
    row_html = BeautifulSoup(res.data, 'html.parser')
    
    # Return elements that matched the html class in a list
    All_leagues = row_html.find_all('main' ,id= "mainContent")
    
    #Split All_championnat to get each championship
    by_league = str(All_leagues).split('<div class="tableCompetitionExplainedContainer"></div>')
    
    #Return elements and class for the current championship (2023)
    Premier_league_23= by_league[0]
    
    return Premier_league_23

In [ ]:
### Let's test the function n°1

Premier_league_23= get_page(urlpage,
                        'main', "mainContent")
print(Premier_league_23)

In [24]:
#### Let's define a function to get informations on each team (function n°2)

def extract_team_23_stats(Premier_league_23, team):
    team_real= team.title()
    pattern = r'data-filtered-table-row-name="{}"'.format(team_real)
    match = re.search(pattern, Premier_league_23)
    start= match.start()
    end = Premier_league_23.index('<tr class="expandable"', start)
    team_data_23 = Premier_league_23[start:end]
    
    position = int(re.findall('data-position="(.*?)">', str(team_data_23))[0])
    data =  re.findall(r'<td.*?>(\d+)<\/td>', team_data_23)
    match_played =int(data[0].strip('<td>').strip('</td>'))
    points = int(re.search(r'<td class="points">\d+</td>', team_data_23).group().strip('<td class="points">').strip('</td>'))
    wins = int(data[1].strip('<td>').strip('</td>'))
    loses = int(data[3].strip('<td>').strip('</td>'))
    drawns = int(data[2].strip('<td>').strip('</td>'))
    goals_for= int(data[4].strip('<td>').strip('</td>'))
    goals_against= int(data[5].strip('<td>').strip('</td>'))

    team_stats = {'position': position,'match_played': match_played,'points': points,'wins': wins,'loses': loses,'drawns': drawns,'goals_for': goals_for,
        'goals_against': goals_against
    }

    return team_stats

In [ ]:
### Let's test the function n°2

extract_team_23_stats(Premier_league_23, "manchester city")

In [26]:
## Let's define our second urlpage for injured and suspended players

urlpage_2 = 'https://www.foot-direct.com/angleterre/premier-league/blessures-et-suspensions'

In [27]:
### Let's define a function to get the web page and subtract the part of the html codes we need (function n°3)

def get_page(urlpage_2,element,html_class):
    # Get page in html
    req_2 = urllib3.PoolManager()
    res_2 = req_2.request('GET', urlpage_2)
    row_html_2 = BeautifulSoup(res_2.data, 'html.parser')
    
    # Return elements that matched the html class in a list
    injured_content = row_html_2.find_all(element , class_= html_class)
    return(injured_content)

In [ ]:
#Let's test the function n°3

injured_content= get_page(urlpage_2, 'div', "content")
print(injured_content)

In [29]:
## First create a list for all teams with injured players
teams = re.findall('<a class="flex--center2" href="/equipe/(.*?)">', str(injured_content))
    
### If the name of the teamm is composed by "-", replace it by espace
for i in range(len(teams)):
    teams[i]= teams[i].replace("-", " ")

In [30]:
#Let's define a function to get all injured players for each team (function n°4)

def player_injured_by_team (team_name):
    team_name= team_name.lower()
       
    ## If teams not in the teams's list, return "team is not in the list"
    
    if team_name not in teams:
        print("team is not in the list")
    
    ## Else, create a row list for each team and apply the boucle while to obtain in a list, all injured players
    
    else:
        player_injured_team= dict() ## To create the dictionary
        team_injured_content = str(injured_content).split('<div class="card card--team marginTop--2')[1:]
        
        players_injured= []
        i =-1
        
        while len(players_injured) < len(team_injured_content):
            i+=1
            player_injured_each_team = re.findall('<a href="/joueur/(.*?)">', str(team_injured_content[i]))
            players_injured.append(player_injured_each_team)
        numero= teams.index(team_name)
        
        ## Dictionnary for the injured players of a team
        player_injured_team.update({teams[numero] : players_injured[numero]})
        return player_injured_team

In [ ]:
# Let's test the function n°4
player_injured_by_team ('Arsenal')

In [34]:
urlpage_3= 'https://fr.besoccer.com/competition/classement/premier/2022'

In [35]:
### Let's define a function to get the web page and subtract the part of the html codes we need (function n°4)

def get_page(urlpage_3,element,html_class):
    # Get page in html
    req_3 = urllib3.PoolManager()
    res_3 = req_3.request('GET', urlpage_3)
    row_html_3 = BeautifulSoup(res_3.data, 'html.parser')
    
    # Return elements that matched the html class in a list
    Premier_league_22 = row_html_3.find_all(element , class_= html_class)
    return(Premier_league_22)

In [ ]:
#Let's test the function n°4

Premier_league_22= str(get_page(urlpage_3, 'tr', 'row-body'))
print(Premier_league_22)

In [37]:
###
def extract_team_22_stats(Premier_league_22, team):
    teams = re.findall('<span class="team-name">(.*?)</span>', str(Premier_league_22))
    position= (teams.index(team)+1)
    start = Premier_league_22.find(team)
    end = Premier_league_22.index("</tr>", start)
    team_data_22 = Premier_league_22[start:end]
    
    match_played= 38
    data = [int(s) for s in re.findall(r'<td.*?>(\d+)</td>', team_data_22)]
    points= data[0]
    wins= data [1]
    drawns= data [2]
    loses =data [3]
    goals_for = data [4]
    goals_against = data [5]

    team_stats22 = {'match_played': match_played,'position': position,'points': points,
                    'wins': wins,'loses': loses ,'drawns':  drawns,'goals_for': goals_for,
        'goals_against':goals_against
    }

    return team_stats22

In [ ]:
## Let's test function 5
extract_team_22_stats(Premier_league_22, "Arsenal")

In [40]:
urlpage_4= 'https://fr.besoccer.com/competition/classement/premier/2021'

In [41]:
### Let's define a function to get the web page and subtract the part of the html codes we need (function n°5)

def get_page(urlpage_4,element,html_class):
    # Get page in html
    req_4 = urllib3.PoolManager()
    res_4 = req_4.request('GET', urlpage_4)
    row_html_4 = BeautifulSoup(res_4.data, 'html.parser')
    
    # Return elements that matched the html class in a list
    Premier_league_21 = row_html_4.find_all(element , class_= html_class)
    return(Premier_league_21)

In [ ]:
#Let's test the function n°5

Premier_league_21= str(get_page(urlpage_4, 'tr', 'row-body'))
print(Premier_league_21)

In [43]:
###
def extract_team_21_stats(Premier_league_21, team):
    teams = re.findall('<span class="team-name">(.*?)</span>', str(Premier_league_21))
    position= (teams.index(team)+1)
    start = Premier_league_21.find(team)
    end = Premier_league_21.index("</tr>", start)
    team_data_21 = Premier_league_21[start:end]
    
    match_played= 38
    data = [int(s) for s in re.findall(r'<td.*?>(\d+)</td>', team_data_21)]
    points= data[0]
    wins= data [1]
    drawns= data [2]
    loses =data [3]
    goals_for = data [4]
    goals_against = data [5]

    team_stats21 = {'match_played': match_played,'position': position,'points': points,
                    'wins': wins,'loses': loses ,'drawns':  drawns,'goals_for': goals_for,
        'goals_against':goals_against
    }

    return team_stats21

In [ ]:
## Let's test function 6
extract_team_21_stats(Premier_league_21, "Arsenal")

In [45]:
urlpage_5= 'https://fr.besoccer.com/competition/classement/premier/2020'

In [46]:
### Let's define a function to get the web page and subtract the part of the html codes we need (function n°7)

def get_page(urlpage_5,element,html_class):
    # Get page in html
    req_5 = urllib3.PoolManager()
    res_5 = req_5.request('GET', urlpage_5)
    row_html_5 = BeautifulSoup(res_5.data, 'html.parser')
    
    # Return elements that matched the html class in a list
    Premier_league_20 = row_html_5.find_all(element , class_= html_class)
    return(Premier_league_20)

In [ ]:
#Let's test the function n°7

Premier_league_20= str(get_page(urlpage_5, 'tr', 'row-body'))
print(Premier_league_20)

In [48]:
###
def extract_team_20_stats(Premier_league_20, team):
    teams = re.findall('<span class="team-name">(.*?)</span>', str(Premier_league_20))
    position= (teams.index(team)+1)
    start = Premier_league_20.find(team)
    end = Premier_league_20.index("</tr>", start)
    team_data_20 = Premier_league_20[start:end]
    
    match_played= 38
    data = [int(s) for s in re.findall(r'<td.*?>(\d+)</td>', team_data_20)]
    points= data[0]
    wins= data [1]
    drawns= data [2]
    loses =data [3]
    goals_for = data [4]
    goals_against = data [5]

    team_stats20 = {'match_played': match_played,'position': position,'points': points,
                    'wins': wins,'loses': loses ,'drawns':  drawns,'goals_for': goals_for,
        'goals_against':goals_against
    }

    return team_stats20

In [49]:
## Let's test function 6
extract_team_20_stats(Premier_league_20, "Arsenal")

{'match_played': 38,
 'position': 8,
 'points': 56,
 'wins': 14,
 'loses': 10,
 'drawns': 14,
 'goals_for': 56,
 'goals_against': 48}